# Azure AI Agents with Streaming

## Overview

This notebook demonstrates how to integrate **Azure AI Agents** into workflows with **streaming capabilities**. We'll create a two-agent pipeline where:
1. **Writer Agent** - Generates content
2. **Reviewer Agent** - Critiques and provides feedback

### Key Concepts:

- **Azure AI Agent Service**: Using Azure-hosted agents with the Agent Framework
- **Streaming Agent Responses**: Observing incremental token generation via `AgentRunUpdateEvent`
- **Workflow Adaptation**: Agents automatically adapt to streaming vs non-streaming modes
- **Output Configuration**: Using `output_response=True` to emit final agent responses
- **Async Context Management**: Properly handling async resources with `AsyncExitStack`

### Features Demonstrated:

✅ Automatic streaming of agent deltas via `AgentRunUpdateEvent`
✅ Adding agents with `WorkflowBuilder.add_agent()` and custom settings
✅ Agents adapt to workflow mode: `run_stream()` emits incremental updates, `run()` emits complete responses
✅ Final `AgentRunResponse` as workflow output

### Prerequisites:

- ✅ Azure AI Agent Service configured
- ✅ Required environment variables set (see below)
- ✅ Azure CLI authentication (`az login` completed)
- ✅ Basic familiarity with WorkflowBuilder and streaming

## Environment Variables Required

Ensure these environment variables are set:

```bash
AZURE_AI_PROJECT_CONNECTION_STRING=<your-connection-string>
# Or alternatively:
AZURE_AI_PROJECT_ENDPOINT=<your-endpoint>
AZURE_AI_PROJECT_ID=<your-project-id>
```

## Import Required Libraries

In [ ]:
import asyncio
from collections.abc import Awaitable, Callable
from contextlib import AsyncExitStack
from typing import Any

from agent_framework import AgentRunUpdateEvent, WorkflowBuilder, WorkflowOutputEvent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv('../../.env')


## Create Azure AI Agent Factory

### Async Context Management Pattern

This helper function demonstrates proper async resource management:
- Uses `AsyncExitStack` to manage multiple async context managers
- Creates `AzureCliCredential` for authentication
- Initializes `AzureAIAgentClient` for agent creation
- Returns a factory function and cleanup function

### Why This Pattern?

Azure AI Agent Client requires async context management. This pattern ensures:
- Proper resource initialization
- Cleanup on completion
- Reusable agent creation

In [ ]:
async def create_azure_ai_agent() -> tuple[Callable[..., Awaitable[Any]], Callable[[], Awaitable[None]]]:
    """Helper method to create an Azure AI agent factory and a close function.

    This makes sure the async context managers are properly handled.
    """
    stack = AsyncExitStack()
    cred = await stack.enter_async_context(AzureCliCredential())

    client = await stack.enter_async_context(AzureAIAgentClient(async_credential=cred))

    async def agent(**kwargs: Any) -> Any:
        return await stack.enter_async_context(client.create_agent(**kwargs))

    async def close() -> None:
        await stack.aclose()

    return agent, close

print("✅ Azure AI Agent factory function defined")

## Initialize Agent Factory

Create the agent factory and cleanup function that we'll use throughout this notebook.

In [ ]:
# Create the agent factory
agent, close = await create_azure_ai_agent()

print("✅ Azure AI Agent factory initialized")
print("   Ready to create agents")

## Create Writer Agent

### Agent Configuration

The Writer Agent is configured with:
- **Name**: "Writer" - for identification in events
- **Instructions**: Defines the agent's role and behavior

In [ ]:
writer = await agent(
    name="Writer",
    instructions=(
        "You are an excellent content writer. You create new content and edit contents based on the feedback."
    ),
)

print("✅ Writer Agent created")

## Create Reviewer Agent

### Agent Configuration

The Reviewer Agent is configured to:
- Provide actionable feedback
- Be concise in feedback delivery
- Evaluate content quality

In [ ]:
reviewer = await agent(
    name="Reviewer",
    instructions=(
        "You are an excellent content reviewer. "
        "Provide actionable feedback to the writer about the provided content. "
        "Provide the feedback in the most concise manner possible."
    ),
)

print("✅ Reviewer Agent created")

## Build the Workflow

### Using `add_agent()` Method

The `WorkflowBuilder.add_agent()` method provides enhanced control:

**Parameters:**
- `id` - Executor identifier for the agent
- `output_response` - When `True`, emits final `AgentRunResponse` as workflow output

**Agent Adaptation:**
- With `run_stream()`: Agents emit incremental `AgentRunUpdateEvent` chunks
- With `run()`: Agents emit complete responses

### Workflow Structure:

```
Writer → Reviewer (outputs final response)
```

In [ ]:
# Build workflow with custom agent settings
workflow = (
    WorkflowBuilder()
    .add_agent(writer, id="Writer")
    .add_agent(reviewer, id="Reviewer", output_response=True)  # Emit final response as output
    .set_start_executor(writer)
    .add_edge(writer, reviewer)
    .build()
)

print("✅ Workflow built successfully!")
print("   Writer → Reviewer (with output_response=True)")

## Run the Workflow with Streaming

### Streaming Execution

This demonstrates:
- **Real-time token streaming** via `AgentRunUpdateEvent`
- **Incremental response display** as agents generate content
- **Final workflow output** containing the complete response

### Event Handling:

- `AgentRunUpdateEvent` - Individual token/chunk from agent
- `WorkflowOutputEvent` - Final complete response from terminal agent

### Display Strategy:

- Track which agent is currently streaming
- Print agent name on first chunk
- Stream tokens inline for real-time feedback

In [ ]:
# User message
user_message = "Create a slogan for a new electric SUV that is affordable and fun to drive."

print(f"\n📝 User Request: {user_message}\n")
print("=" * 70)
print("\n🔄 Streaming Agent Responses:\n")

# Track which agent is currently streaming
last_executor_id: str | None = None

# Stream the workflow execution
events = workflow.run_stream(user_message)

async for event in events:
    if isinstance(event, AgentRunUpdateEvent):
        eid = event.executor_id
        
        # Print agent name when switching to a new agent
        if eid != last_executor_id:
            if last_executor_id is not None:
                print()  # New line after previous agent
            print(f"\n🤖 {eid}:", end=" ", flush=True)
            last_executor_id = eid
        
        # Stream token inline
        print(event.data, end="", flush=True)
    
    elif isinstance(event, WorkflowOutputEvent):
        print("\n\n" + "=" * 70)
        print("\n📤 Final Workflow Output:\n")
        print(event.data)

print("\n" + "=" * 70)
print("\n✅ Workflow execution completed!")

## Cleanup Resources

Always clean up async resources properly to avoid resource leaks.

In [ ]:
# Close all async resources
await close()

print("✅ Resources cleaned up successfully")

## Expected Output

### Sample Streaming Execution:

```
🤖 Writer: Charge Up Your Adventure—Affordable Fun, Electrified!

🤖 Reviewer: Slogan: "Plug Into Fun—Affordable Adventure, Electrified."

**Feedback:**
- Clear focus on affordability and enjoyment
- "Plug into fun" emotionally connects and highlights electric nature
- Consider specifying "SUV" for clarity
- Strong, upbeat marketing tone

======================================================================
📤 Final Workflow Output:

AgentRunResponse(...)
```

### How It Works:

1. **Writer Agent** receives user message
2. Writer streams content creation token-by-token
3. Complete writer response flows to **Reviewer Agent**
4. Reviewer streams feedback token-by-token
5. Final `AgentRunResponse` emitted as workflow output (due to `output_response=True`)

## Key Takeaways

### Azure AI Agent Service Integration

✅ **Azure-Hosted Agents**
- Fully managed AI agents in Azure
- No local model deployment needed
- Enterprise-grade security and compliance

✅ **Agent Factory Pattern**
- Use `AsyncExitStack` for proper async resource management
- Create reusable agent factory functions
- Always clean up with `close()` function

### Streaming vs Non-Streaming

| Feature | `run()` | `run_stream()` |
|---------|---------|----------------|
| **Agent Behavior** | Complete response | Token-by-token streaming |
| **Events** | Final results only | `AgentRunUpdateEvent` chunks |
| **User Experience** | Wait for completion | Real-time feedback |
| **Best For** | Batch processing | Interactive applications |

### WorkflowBuilder.add_agent() Configuration

**Basic Usage:**
```python
.add_agent(agent, id="agent_id")
```

**With Output Response:**
```python
.add_agent(agent, id="agent_id", output_response=True)
```
- Emits complete `AgentRunResponse` as workflow output
- Useful for capturing full agent metadata
- Typically used on terminal agents

### Event Types

**AgentRunUpdateEvent:**
- Fired during streaming execution
- Contains individual token/chunk
- Has `executor_id` to identify source agent
- Allows real-time response display

**WorkflowOutputEvent:**
- Contains final workflow output
- Fired when terminal agent completes
- Can contain `AgentRunResponse` if `output_response=True`

### Async Resource Management

✅ **AsyncExitStack Pattern:**
```python
stack = AsyncExitStack()
resource1 = await stack.enter_async_context(Resource1())
resource2 = await stack.enter_async_context(Resource2())
# ... use resources ...
await stack.aclose()  # Cleans up all resources
```

**Benefits:**
- Manages multiple async context managers
- Ensures proper cleanup order
- Prevents resource leaks

### Comparison: Azure AI vs Azure OpenAI Agents

| Feature | Azure AI Agents | Azure OpenAI Agents |
|---------|----------------|---------------------|
| **Service** | Azure AI Agent Service | Azure OpenAI Service |
| **Client** | `AzureAIAgentClient` | `AzureOpenAIChatClient` |
| **Streaming** | `AgentRunUpdateEvent` | `AgentRunEvent` + streaming |
| **Management** | Async context required | Direct instantiation |
| **Use Case** | Enterprise AI agents | OpenAI-specific features |

### Next Steps

Explore more agent patterns:
- **Function Bridge**: Using tools and function calling
- **Tool Calls with Feedback**: Interactive tool execution
- **Custom Agent Executors**: Building specialized agent behaviors
- **Human-in-the-Loop**: Workflows requiring user approval